In [1]:

import os
import warnings
import sys

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

In [2]:
!pip install mlflow


In [3]:
data_path = "data/wine-quality.csv"
data = pd.read_csv(data_path)

data.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
652,7.1,0.26,0.34,14.40,0.067,35.0,189.0,0.99860,3.07,0.53,9.1,7
3215,7.1,0.14,0.33,1.00,0.104,20.0,54.0,0.99057,3.19,0.64,11.5,6
2701,6.5,0.23,0.36,16.30,0.038,43.0,133.0,0.99924,3.26,0.41,8.8,5
1286,6.6,0.16,0.29,1.80,0.050,40.0,147.0,0.99120,3.06,0.44,11.4,7
3077,6.2,0.25,0.38,7.90,0.045,54.0,208.0,0.99572,3.17,0.46,9.1,5
811,6.0,0.41,0.05,1.50,0.063,17.0,120.0,0.99320,3.21,0.56,9.2,6
2516,7.1,0.32,0.29,4.00,0.038,33.0,170.0,0.99463,3.27,0.64,10.2,6
1263,9.6,0.29,0.46,1.45,0.039,77.5,223.0,0.99440,2.92,0.46,9.5,6
4552,6.4,0.55,0.26,9.60,0.027,20.0,104.0,0.99240,3.22,0.73,13.1,6
3797,7.0,0.39,0.21,10.70,0.098,13.0,91.0,0.99657,3.03,0.47,9.3,5


In [4]:
remote_server_uri = "http://0.0.0.0:5000" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)

In [5]:
mlflow.tracking.get_tracking_uri()
# to check where the server is running


'http://0.0.0.0:5000'

In [7]:
exp_name = "ElasticNet_wine"
mlflow.set_experiment(exp_name)

2024/04/07 18:28:28 INFO mlflow.tracking.fluent: Experiment with name 'ElasticNet_wine' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/swetha/MLFLOW/mlruns/953650410507073103', creation_time=1712494708302, experiment_id='953650410507073103', last_update_time=1712494708302, lifecycle_stage='active', name='ElasticNet_wine', tags={}>

In [8]:
def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


def load_data(data_path):
    data = pd.read_csv(data_path)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, train_y, test_x, test_y


def train(alpha=0.5, l1_ratio=0.5):
    # train a model with given parameters
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    data_path = "data/wine-quality.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        
        mlflow.sklearn.log_model(lr, "model")

In [9]:
train(0.5, 0.5)


Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160693
  R2: 0.12678721972772689
Save to: /home/swetha/MLFLOW/mlruns/953650410507073103/abf6642b709d448189a4c527f00cf531/artifacts


In [10]:
train(0.2, 0.2)


Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7859129997062341
  MAE: 0.6155290394093893
  R2: 0.20224631822892103
Save to: /home/swetha/MLFLOW/mlruns/953650410507073103/86de8c952a03417d8c94b1e30fa8808e/artifacts


In [11]:
train(0.1, 0.1)


Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7792546522251949
  MAE: 0.6112547988118587
  R2: 0.2157063843066196
Save to: /home/swetha/MLFLOW/mlruns/953650410507073103/d4d21c4dd3ad4ab387b36164067a98d1/artifacts
